In [ ]:
import numpy as np
import pandas as pd
import sys
import os 
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
import glob

from sglm import utils, glm_fit

## Create a project

#### First, let's create a new project. The project directory will create a data and results folder and a config file.

#### You will need to edit the config file with the particular glm params you wish to use. Fields that are necessary to edit are: predictors, predictors_shift_bounds, response, and the glm_keyword_args.

#### You will also need to move your data into the data folder.

In [ ]:
project_name = 'test_glm'
project_dir = r'C:\Users\janet\Documents\orphan_code'

utils.create_new_project(project_name, project_dir)

# Import and Format Data

Input data should conform to the following convention and be saved as a *.csv:

Indices / Unique Row Identifiers:
* SessionName -- Any order is acceptable
* TrialNumber-- Must be in chronological order, but does not need to start from zero
* Timestamp -- Must be in chronological order, but does not need to start from zero

Columns (Predictors + Responses):
* Predictors - binary
* Reponses - e.g. neural responses (analog or binary)

Example, shown below is dummy data depicting a trial_0 that last four response timestamps:
| SessionName | TrialNumber | Timestamp | predictor_1 | predictor_2 | predictor_3 | response_1 | response_2 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| session_0 | trial_0 | -1 | 0 | 0 | 0 | 1 | 0.3 |
| session_0 | trial_0 | 0 | 0 | 0 | 0 | 0 | 1.4 |
| session_0 | trial_0 | 1 | 0 | 0 | 0 | 1 | 2.3 |
| session_0 | trial_0 | 2 | 0 | 1 | 0 | 1 | 0.3 |
| session_0 | trial_1 | -2 | 0 | 0 | 0 | 0 | 1.4 |
| session_0 | trial_1 | -1 | 0 | 0 | 0 | 1 | 2.3 |
| session_0 | trial_1 | 0 | 1 | 0 | 0 | 0 | 1.4 |
| session_0 | trial_1 | 1 | 0 | 0 | 0 | 1 | 2.3 |
| session_1 | trial_0 | 5 | 0 | 0 | 0 | 0 | 1.4 |
| session_1 | trial_0 | 6 | 1 | 0 | 0 | 1 | 2.3 |
| session_1 | trial_0 | 7 | 0 | 0 | 0 | 0 | 1.4 |
| session_1 | trial_0 | 8 | 0 | 0 | 0 | 1 | 2.3 |
| session_1 | trial_1 | 9 | 0 | 0 | 0 | 0 | 1.4 |
| session_1 | trial_1 | 10 | 0 | 0 | 0 | 1 | 2.3 |
....

#### Now, let's get set up to start our project

In [ ]:
project_path = os.path.join(project_dir, project_name)
files = os.listdir(project_path)

assert 'data' in files, 'data folder not found! {}'.format(files)
assert 'results' in files, 'results folder not found! {}'.format(files)
assert 'config.yaml' in files, 'config.yaml not found! {}'.format(files)

#### If needed, use the following function to combine multiple sessions into one csv. You will need a filename you wish to call your output_csv

In [ ]:
output_csv = 'combined.csv'

utils.combine_csvs(project_path, output_csv)

#### Next, we'll open the data and set the columns you wish to use as fixed indices

In [ ]:
input_file = os.path.join(project_path, 'data', output_csv)
index_col = ['SessionName', 'TrialNumber', 'Timestamp']

df = utils.read_data(input_file, index_col)

print('Your dataframe has {} rows and {} columns'.format(df.shape[0], df.shape[1]))

#### You can now explore and add to the dataframe. As an example, you may want to add various "predictors" or "features" to explore. You can use the example below as inspiration

In [ ]:
#Identify the individual licks that have specific meaning in the tasks: 
#lick 1, lick 2 and lick 3 are "operant licks" on different training days
#licknon1-3 are all the other licks

df_source = df.copy()
srs_lick = df_source.groupby(['SessionName', 'TrialNumber'])['lick'].cumsum()
srs_lick_count = srs_lick * df_source['lick']
df_lick_count_dummies = pd.get_dummies(srs_lick_count).drop(0, axis=1)
df_lick_count_dummies = df_lick_count_dummies[[1,2,3]]
df_lick_count_dummies['non1-3'] = df_source['lick'] - df_lick_count_dummies.sum(axis=1)
df_lick_count_dummies.columns = [f'lick_{original_column_name}' for original_column_name in df_lick_count_dummies.columns]

# Columns lick and lick_1, lick_2, lick_3, lick_non-13 should not all be used together
# as predictors because of multicollinearity.
df_source = pd.concat([df_source, df_lick_count_dummies], axis=1)
df_source

assert np.all(df_source['lick'] == df_source[['lick_1', 'lick_2', 'lick_3', 'lick_non1-3']].sum(axis=1)), 'Column lick should equal the sum of all other lick columns.'

#### Friendly reminder, the df we have imported is mutli-index, meaning, it's organization is dependent on 3-columns that we have set in index_col. Therefore, we can use "groupby" if you need to split the organization. 

In [ ]:
reIndex = df_source.groupby(level=[0, 1])

## Load your fitting paramaters and set up your train/test data

In [ ]:
config_file = os.path.join(project_path, 'config.yaml')
config = utils.load_config(config_file)

#### Shift responses and predictors. If you do not want to shift your predictors by an amount you set, feel free to comment out the entire "predictors_shift_bounds" in config.yaml. We will then use the default set when we created the config file.

In [ ]:
response_shift, df_predictors_shift, shifted_params = glm_fit.shift_predictors(config, df)
print('Your dataframe was shifted using: {}'.format(shifted_params))

### Create your test/train datasets

In [ ]:
X_train,X_test, y_train, y_test = glm_fit.split_data(df_predictors_shift, response_shift, config)

print('Training data has {} rows and {} columns'.format(X_train.shape[0], X_train.shape[1]))
print('Testing data has {} rows and {} columns'.format(X_test.shape[0], X_test.shape[1]))

## Now, we're ready to run our GLM!

### We have two different options: ElasticNet and Ridge.

#### ElasticNet: This is a linear regression model trained with both L1 and L2 prior as regularizer. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge. We will use the glm_fit.fit_tuned_EN function to tune the hyperparams and then fit the model. If you know which params you would like to use, you can use the glm_fit.fit_EN function.

In [ ]:
# Fit the ElasticNet model
model, y_pred, score, beta, intercept, sparse_beta = glm_fit.fit_EN(config, X_train, X_test, y_train, y_test)
print('Your model can account for {} percent of your data'.format(score*100))

In [ ]:
# Fit the ElasticNet model with cross validation: remember, your alphas and l1_ratios should be lists
tuned_model, y_pred, score, beta, best_params = glm_fit.fit_tuned_EN(config, X_train, X_test, y_train, y_test)
print('Your model can account for {} percent of your data'.format(score*100))

#### Ridge: This is a linear regression model trained with L2 prior as regularizer. This is the standard Linear Regression you're familiar with. We will use the glm_fit.fit_tuned_ridge function to tune the hyperparams and then fit the model. If you know which params you would like to use, you can use the glm_fit.fit_ridge function. 

In [ ]:
# Fit the Ridge model
ridge_model, y_pred, score, beta, intercept = glm_fit.fit_ridge(config, X_train, X_test, y_train, y_test)
print('Your model can account for {} percent of your data'.format(score*100))

In [ ]:
# Fit the Ridge model with cross validation: remember, your alphas should be a list
tuned_ridge_model, y_pred, score, beta, best_params = glm_fit.fit_tuned_ridge(config, X_train, X_test, y_train, y_test)

## Save your outputs

In [ ]:
#Create your model dictonary, this should include all the information you wish to save
model_dict = {'model': model,
              'model_type': 'ElasticNet', #or 'Ridge'
              'y_pred': y_pred,
              'score': score,
              'beta': beta,
              'intercept': intercept,
              'sparse_beta': sparse_beta,}

#Save your model dictionary
glm_fit.save_model(model_dict, config)

## Generate and save figures

In [ ]:
glm_fit.plot_and_save(config, y_pred, y_test, beta, df_predictors_shift)